In [1]:
from sentence_transformers import SentenceTransformer

/home/narayan/Documents/awesome scripts/SU Chatbot/su_chatBot/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
import pandas as pd
df = pd.read_csv('./SUCHAT_EMB - Sheet1.csv')
questions = df["question"]
answers = df["answer"]

In [3]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/home/narayan/Documents/awesome scripts/SU Chatbot/su_chatBot/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
question_embeddings = model.encode(questions)

In [5]:
import psycopg2
from psycopg2 import sql

# Connect to your PostgreSQL database
def connect_to_database():
    conn = psycopg2.connect(
    dbname="su_chat_bot",
    user="postgres",
    password="love123",
    host="localhost",
    port="5432"
    )

    return conn

In [ ]:
# Create a cursor object
conn = connect_to_database()
cur = conn.cursor()

# Create a table
create_table_query = """
CREATE TABLE IF NOT EXISTS question_answers (
    id SERIAL PRIMARY KEY,
    question TEXT NOT NULL,
    answer TEXT NOT NULL,
    question_embeddings VECTOR(384) -- Assuming 384 dimensions for vector
);
"""
cur.execute(create_table_query)
conn.commit()

insert_query = """
INSERT INTO question_answers (question, answer, question_embeddings)
VALUES (%s, %s, %s);
"""

for i in range(len(questions)):
    question = questions[i]
    answer = answers[i]
    embeddings = question_embeddings[i]
    cur.execute(insert_query, (question, answer, embeddings.tolist()))
    conn.commit()

cur.close()
conn.close()

Question answering session

In [7]:
conn = connect_to_database()
cur = conn.cursor()
query = '''
SELECT answer, 
       question_embeddings <=> %s AS similarity
FROM question_answers
ORDER BY similarity DESC
LIMIT 1;
'''

In [15]:
# get connection
conn = connect_to_database()
cur = conn.cursor()
query = '''
    SELECT answer 
    FROM question_answers
    ORDER BY question_embeddings <=> %s::vector
    LIMIT 1;
    '''
def answer_question(question):
    asked_question_embeddings = model.encode(question).astype(float).tolist()
    cur.execute(query, (asked_question_embeddings,))
    results = cur.fetchone()
    return results[0]

In [21]:
question = input("Enter your question here: ")
answer = answer_question(question)
print("Answer:", answer)

Answer: We are living in unprecedented times where software has transformed how we live. From smartphones to smart wearables, from finding information to buying things, from banking to healthcare, from listening to music to driving a car, every aspect of our life is impacted by computers. This trend will only accelerate with the new age of AI and Machine Learning. The future belongs to nations who can make fundamental advances in Computer Science. To ensure that the brightest minds from underprivileged backgrounds in the world get an opportunity to study Computer Science, leaders from Silicon Valley, top American and European institutions, industry leaders, and venture capitalists have come together to build a new University focussed on Computer Science - Sitare University. The first Sitare batch was started in 2022 with twenty three students, and the second batch admitted in 2023 has forty four students. We plan to admit around 150 students in 2024, and gradually increase this number 